## 1. 环境配置与路径设置

In [ ]:
import os
import sys
import glob
import json
import warnings
warnings.filterwarnings('ignore')

# 🚀 指定使用 4, 5, 6, 7 四张 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

# 🛠️ 设置项目根目录
PROJECT_ROOT = Path("/home/test/ybk/nids/encoder/PCAP_encoder")
os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

print(f"📂 项目根目录: {PROJECT_ROOT}")
# ⚠️ 注意：为了多卡并行，此处严禁调用 torch.cuda.is_available() 或任何触碰 GPU 的操作
print(f"📌 当前使用的 GPU 索引: {os.environ.get('CUDA_VISIBLE_DEVICES')}")
print(f"📦 PyTorch 版本: {torch.__version__}")

📂 项目根目录: /home/test/ybk/nids/encoder/PCAP_encoder
📌 当前使用的 GPU 索引: 4,5,6,7
📦 PyTorch 版本: 2.2.2+cu118


## 2. 导入核心模块

我们将直接调用原作者定义的 `Core` 模块，确保实验逻辑与论文一致。

In [12]:
from Core.classes.dataset_for_classification import Classification_Dataset
from Core.classes.classification_model import Classification_model
from Core.classes.tokenizer import QA_Tokenizer_T5
from Core.classes.logger import TrainingExperimentLogger

print("✅ 核心模块导入成功!")

✅ 核心模块导入成功!


## 3. 实验参数配置

在这里我们配置微调的关键参数。注意：为了获得更好的效果，我们**解冻了编码器**并降低了学习率。

In [13]:
# ============================================================
# 📋 核心配置 - 针对 NIDS 微调优化
# ============================================================

# 1. 路径配置
DATA_DIR = PROJECT_ROOT / "data" / "CIC-IDS2017" / "Classification"
TRAINING_DATA = str(DATA_DIR / "train.parquet")
VALIDATION_DATA = str(DATA_DIR / "val.parquet")
TESTING_DATA = str(DATA_DIR / "test.parquet")

# 预训练权重路径 (weights.pth 所在目录)
PRETRAINED_MODEL_PATH = str(PROJECT_ROOT / "models")

# 2. 训练策略 (完全解冻模式)
FIX_ENCODER = False      # ❌ False 表示解冻编码器进行全量微调
LEARNING_RATE = 2e-5     # 📉 解冻时使用极小的学习率 (2e-5)
EPOCHS = 15              # 训练轮数
BATCH_SIZE = 16          # 批次大小 (根据显存调整)
LOSS_TYPE = "weighted"   # ⚖️ 使用加权损失处理类别不平衡

# 3. 模型架构
MODEL_NAME = "t5-base"   # 必须与预训练权重匹配
TOKENIZER_NAME = "t5-base"
BOTTLENECK = "mean"      # 提取特征的方式
MAX_QST_LENGTH = 512
MAX_ANS_LENGTH = 32
SEED = 42

print(f"🚀 实验模式: {'全量微调 (Unfrozen)' if not FIX_ENCODER else '线性探测 (Frozen)'}")
print(f"🔗 学习率: {LEARNING_RATE}")

🚀 实验模式: 全量微调 (Unfrozen)
🔗 学习率: 2e-05


## 4. 数据预处理

将原始的 CIC-IDS2017 字节数据转换为模型可读的十六进制文本格式。

In [14]:
def bytes_to_hex(byte_array: np.ndarray, format_type: str = 'every4') -> list:
    """将字节数组转换为十六进制字符串"""
    hex_strings = []
    for row in tqdm(byte_array, desc="🔄 转换十六进制", leave=False):
        hex_str = ''.join(f'{int(b):02x}' for b in row)
        if format_type == 'every4':
            hex_str = ' '.join(hex_str[i:i+4] for i in range(0, len(hex_str), 4))
        hex_strings.append(hex_str.strip())
    return hex_strings

def load_raw_nids_data(input_dir, num_files=5):
    """
    🚀 慢速步骤：从磁盘加载原始 Parquet 分片到内存
    """
    input_dir = Path(input_dir)
    files = sorted(glob.glob(str(input_dir / "Payload_Bytes_File_*.parquet")))
    
    print(f"⏳ 正在从 {input_dir} 加载前 {num_files} 个分片...")
    dfs = []
    for f in tqdm(files[:num_files], desc="📂 加载 Parquet 分片"):
        dfs.append(pd.read_parquet(f))
    
    df_all = pd.concat(dfs, ignore_index=True)
    print(f"✅ 原始数据加载完成，共 {len(df_all)} 条记录。")
    return df_all

# 1. 执行加载 (这一步只需要运行一次)
RAW_PAYLOAD_DIR = PROJECT_ROOT / "data" / "CIC-IDS2017" / "Payload-Bytes"
df_raw = load_raw_nids_data(RAW_PAYLOAD_DIR, num_files=5)

⏳ 正在从 /home/test/ybk/nids/encoder/PCAP_encoder/data/CIC-IDS2017/Payload-Bytes 加载前 5 个分片...


📂 加载 Parquet 分片: 100%|██████████| 5/5 [01:03<00:00, 12.79s/it]


✅ 原始数据加载完成，共 8378784 条记录。


In [15]:
def process_and_save_nids_data(df, output_dir, sample_size=20000):
    """
    ⚡ 改进版：平衡采样策略
    """
    output_dir = Path(output_dir)
    
    # 1. 统计原始分布
    print(f"📊 原始类别分布:\n{df['attack_label'].value_counts()}")
    
    # 2. 平衡采样逻辑
    # 目标：每个类别尽量达到 target_per_class 条样本
    target_per_class = sample_size // df['attack_label'].nunique()
    balanced_dfs = []
    
    for label in df['attack_label'].unique():
        df_group = df[df['attack_label'] == label]
        if len(df_group) >= target_per_class:
            # 多数类：下采样
            balanced_dfs.append(df_group.sample(n=target_per_class, random_state=SEED))
        else:
            # 少数类：过采样 (重复利用现有样本)
            balanced_dfs.append(df_group.sample(n=target_per_class, replace=True, random_state=SEED))
    
    df_balanced = pd.concat(balanced_dfs, ignore_index=True)
    print(f"⚖️ 平衡后总样本数: {len(df_balanced)}")

    # 3. 标签编码
    labels, uniques = pd.factorize(df_balanced['attack_label'], sort=True)
    df_balanced['class'] = labels.astype(np.int64)
    
    # 4. 提取 Payload 并转换十六进制
    print(f"🔄 正在转换 {len(df_balanced)} 条数据的字节为十六进制文本...")
    payload_cols = [f"payload_byte_{i}" for i in range(1, 65)]
    X_bytes = df_balanced[payload_cols].fillna(0).values.astype(np.uint8)
    contexts = bytes_to_hex(X_bytes)
    
    # 5. 构造最终 DataFrame
    new_df = pd.DataFrame({
        'question': 'Classify the network packet',
        'context': contexts,
        'class': df_balanced['class'].values,
        'type_q': df_balanced['attack_label'].values
    })
    
    # 6. 划分数据集 (此时每个类都有足够样本，不会报错)
    print(f"✂️ 正在划分数据集...")
    train_df, temp_df = train_test_split(new_df, test_size=0.3, random_state=SEED, stratify=new_df['class'])
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df['class'])
    
    # 7. 保存
    output_dir.mkdir(parents=True, exist_ok=True)
    train_df.to_parquet(output_dir / "train.parquet", index=False)
    val_df.to_parquet(output_dir / "val.parquet", index=False)
    test_df.to_parquet(output_dir / "test.parquet", index=False)
    
    print(f"✅ 平衡数据处理完成！")
    print(f"📊 最终训练集类别分布:\n{train_df['type_q'].value_counts()}")

# 执行处理：建议增加 sample_size 到 20000 以上
process_and_save_nids_data(df_raw, DATA_DIR, sample_size=20000)

📊 原始类别分布:
attack_label
BENIGN                        7727363
DoS Hulk                       592302
DoS GoldenEye                   23923
Heartbleed                      20920
Web Attack – Brute Force         9437
Web Attack – XSS                 2798
DoS Slowhttptest                 2001
Web Attack – Sql Injection         26
DoS slowloris                      14
Name: count, dtype: Int64
⚖️ 平衡后总样本数: 19998
🔄 正在转换 19998 条数据的字节为十六进制文本...


✂️ 正在划分数据集...
✅ 平衡数据处理完成！
📊 最终训练集类别分布:
type_q
Heartbleed                    1556
Web Attack – Sql Injection    1556
DoS GoldenEye                 1556
Web Attack – XSS              1555
DoS Hulk                      1555
BENIGN                        1555
DoS slowloris                 1555
Web Attack – Brute Force      1555
DoS Slowhttptest              1555
Name: count, dtype: Int64


## 5. 初始化实验组件

我们将逐步初始化分词器、数据集和日志记录器。

In [16]:
# 1. 构建配置字典
opts = {
    "identifier": f"nids_finetune_lr{LEARNING_RATE}_unfrozen_multiGPU",
    "experiment": "CIC-IDS2017_NIDS",
    "task": "supervised",
    "clean_start": True,
    "model_name": MODEL_NAME,
    "tokenizer_name": TOKENIZER_NAME,
    "finetuned_path_model": PRETRAINED_MODEL_PATH,
    "bottleneck": BOTTLENECK,
    "pkt_repr_dim": 768,
    "use_pkt_reduction": False,
    "lr": LEARNING_RATE,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE, # 每张卡的 batch size
    "seed": SEED,
    "loss": LOSS_TYPE,
    "fix_encoder": FIX_ENCODER,
    "training_data": TRAINING_DATA,
    "validation_data": VALIDATION_DATA,
    "testing_data": TESTING_DATA,
    "percentage": 100,
    "max_qst_length": MAX_QST_LENGTH,
    "max_ans_length": MAX_ANS_LENGTH,
    "input_format": "every4",
    "output_path": str(PROJECT_ROOT / "results"),
    "log_level": "info",
    "use_cuda": True, # 🚀 直接设为 True，严禁在此处调用 torch.cuda.is_available()
    "gpu": "cuda", 
}

print("✅ 实验配置已更新为多卡模式!")

✅ 实验配置已更新为多卡模式!


## 7. 执行分布式微调训练

⚠️ **重要提示**: 
1. **必须重启 Kernel**: 由于 `notebook_launcher` 要求在启动前不能有任何 `Accelerator` 实例被初始化，请点击上方菜单的 **"Restart"**。
2. **跳过全局加载**: 我们已经删除了之前的“加载数据与模型”单元格，因为在多卡模式下，每个进程必须在自己的空间内独立初始化模型。
3. **直接运行**: 重启后，依次运行“环境配置”、“实验参数”、“数据预处理”和“构建配置字典”，然后直接运行本单元格。

In [17]:
from accelerate import notebook_launcher

def train_distributed():
    """
    🚀 多卡分布式训练包装函数
    """
    # 1. 初始化日志 (内部会创建 Accelerator)
    local_logger = TrainingExperimentLogger(opts)
    
    # 只在主进程打印初始化信息
    if local_logger.accelerator.is_main_process:
        print(f"📢 主进程启动，正在为 {local_logger.accelerator.num_processes} 张显卡准备环境...")
    
    local_logger.start_experiment(opts)
    
    # 2. 初始化分词器
    local_tokenizer = QA_Tokenizer_T5(opts)
    
    # 3. 加载数据集
    local_dataset_trainval = Classification_Dataset(opts, local_tokenizer)
    local_dataset_trainval.load_dataset("Train", TRAINING_DATA, "every4", VALIDATION_DATA)
    
    local_dataset_test = Classification_Dataset(opts, local_tokenizer)
    local_dataset_test.load_dataset("Test", TESTING_DATA, "every4")
    
    # 4. 初始化模型
    local_model = Classification_model(opts, local_tokenizer, local_dataset_trainval, local_dataset_test)
    
    if local_logger.accelerator.is_main_process:
        print(f"🚀 开始 NIDS 微调训练 (模式: {'全量微调' if not FIX_ENCODER else '仅训练分类头'})")
        print(f"   - 总 Batch Size: {opts['batch_size'] * local_logger.accelerator.num_processes}")
    
    try:
        local_model.run(local_logger, opts)
        if local_logger.accelerator.is_main_process:
            print("\n🎉 训练成功完成！")
    except Exception as e:
        # 打印错误堆栈以方便调试
        import traceback
        print(f"\n❌ 进程 {local_logger.accelerator.process_index} 出错: {e}")
        traceback.print_exc()
    finally:
        local_logger.end_experiment()

# 🚀 启动分布式训练
# 注意：运行此单元格前必须先 Restart Kernel，确保没有全局 Accelerator 实例
# 使用 use_port 参数指定一个未被占用的端口
notebook_launcher(train_distributed, num_processes=4, use_port="29501")

Launching training on 4 GPUs.
📢 主进程启动，正在为 4 张显卡准备环境...

############### Running experiment t5-base_standard-tokenizer_CIC-IDS2017_NIDSnids_finetune_lr2e-05_unfrozen_multiGPU_task-supervised_lr-2e-05_epochs-15_batch-16 for supervised ###############

🚀 开始 NIDS 微调训练 (模式: 全量微调)
   - 总 Batch Size: 64


  7%|▋         | 218/3052 [01:11<2:03:36,  2.62s/it]Deleting 1 checkpoints to make room for new checkpoint.
Removed shared tensor {'lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
 14%|█▍        | 436/3052 [02:14<1:37:07,  2.23s/it]Deleting 1 checkpoints to make room for new checkpoint.
Removed shared tensor {'lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
 21%|██▏       | 654/3052 [03:31<1:32:11,  2.31s/it]Deleting 1 checkpoints to make room for new checkpoint.
Removed shared tensor {'lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
 29%|██▊       | 872/3052 [04:47<1:34:32,  2.60s/it]Deleting 1 checkpoints to make room for new checkpoint.
Removed shared tensor {'lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any 

End training...
Start testing...


100%|██████████| 46/46 [00:04<00:00,  9.53it/s]



🎉 训练成功完成！


[2026-01-04 21:05:40,977] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 954104 via signal SIGTERM
[2026-01-04 21:05:40,980] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 954106 via signal SIGTERM
[2026-01-04 21:05:40,981] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 954163 via signal SIGTERM


## 8. 结果分析

训练完成后，结果将保存在 `results/` 目录下。你可以查看混淆矩阵和 F1 分数。

In [18]:
import glob

# 1. 动态查找权重文件路径 (原作者代码生成的路径非常深)
# 结构: results/supervised/t5-base_standard-tokenizer/实验ID/参数/seed_42/best_model/weights.pth
search_pattern = str(PROJECT_ROOT / "results" / "supervised" / "*" / (opts['experiment'] + opts['identifier']) / "*" / f"seed_{SEED}" / "best_model" / "weights.pth")
found_weights = glob.glob(search_pattern)

if found_weights:
    best_weight_path = Path(found_weights[0])
    results_dir = best_weight_path.parent.parent
    print(f"📂 实验结果根目录: {results_dir}")
    print(f"✅ 最佳权重路径: {best_weight_path}")
    print(f"⚖️ 权重大小: {best_weight_path.stat().st_size / 1024 / 1024:.2f} MB")
    
    # 2. 读取评估结果 (JSON 格式)
    eval_file = PROJECT_ROOT / "evaluation" / f"{opts['experiment']}{opts['identifier']}.json"
    if eval_file.exists():
        print(f"\n📊 评估指标 (来自 {eval_file.name}):")
        with open(eval_file, 'r') as f:
            lines = f.readlines()
            # 最后一行为测试集结果，之前的行为每个 epoch 的验证结果
            test_results = json.loads(lines[-1])
            
            print(f"   - 测试集准确率 (Accuracy): {test_results.get('accuracy', 0)*100:.2f}%")
            print(f"   - Macro F1 Score: {test_results.get('f1_score_macro', 0):.4f}")
            print(f"   - Micro F1 Score: {test_results.get('f1_score_micro', 0):.4f}")
            print(f"   - 测试耗时: {test_results.get('test_time', 0):.2f}s")
    else:
        print(f"⚠️ 未找到评估 JSON 文件: {eval_file}")
else:
    print("⚠️ 未找到权重文件。请检查：")
    print(f"   1. 搜索模式是否正确: {search_pattern}")
    print(f"   2. 训练是否因显存不足等原因中途崩溃。")
    # 尝试列出 results 目录看看结构
    print("\n当前 results 目录结构:")
    os.system(f"ls -R {PROJECT_ROOT / 'results' / 'supervised'} | head -n 20")

📂 实验结果根目录: /home/test/ybk/nids/encoder/PCAP_encoder/results/supervised/t5-base_standard-tokenizer/CIC-IDS2017_NIDSnids_finetune_lr2e-05_unfrozen_multiGPU/task-supervised_lr-2e-05_epochs-15_batch-16/seed_42
✅ 最佳权重路径: /home/test/ybk/nids/encoder/PCAP_encoder/results/supervised/t5-base_standard-tokenizer/CIC-IDS2017_NIDSnids_finetune_lr2e-05_unfrozen_multiGPU/task-supervised_lr-2e-05_epochs-15_batch-16/seed_42/best_model/weights.pth
⚖️ 权重大小: 2.28 MB

📊 评估指标 (来自 CIC-IDS2017_NIDSnids_finetune_lr2e-05_unfrozen_multiGPU.json):
   - 测试集准确率 (Accuracy): 84.92%
   - Macro F1 Score: 0.8512
   - Micro F1 Score: 0.8492
   - 测试耗时: 4.83s


## 9. 可视化分析：混淆矩阵与分类报告

为了更直观地评估模型在各个类别上的表现，我们绘制混淆矩阵并打印详细的分类报告。这有助于识别模型最容易混淆的攻击类型。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import DataLoader

# 1. 获取类别映射 (从测试集文件中读取)
test_df_for_labels = pd.read_parquet(TESTING_DATA)
# 建立 class -> type_q 的映射
class_to_label = test_df_for_labels.drop_duplicates('class').sort_values('class')[['class', 'type_q']]
label_names = class_to_label['type_q'].tolist()

print(f"🏷️ 类别映射: {dict(zip(class_to_label['class'], class_to_label['type_q']))}")

# 2. 加载模型进行推理 (单卡模式即可)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用设备: {device}")

# 初始化分词器
viz_tokenizer = QA_Tokenizer_T5(opts)

# 初始化数据集
viz_dataset_test = Classification_Dataset(opts, viz_tokenizer)
viz_dataset_test.load_dataset("Test", TESTING_DATA, "every4")
viz_test_loader = DataLoader(viz_dataset_test, batch_size=BATCH_SIZE, shuffle=False)

# 初始化模型包装器
viz_model_wrapper = Classification_model(opts, viz_tokenizer, None, viz_dataset_test)

# 加载最佳权重
# 注意：best_weight_path 是在上一单元格中定义的
if 'best_weight_path' in locals():
    print(f"📥 正在加载权重: {best_weight_path}")
    viz_model_wrapper.classification_head.load_state_dict(torch.load(best_weight_path))
    viz_model_wrapper.classification_head.to(device)
    viz_model_wrapper.custom_model.to(device)
    viz_model_wrapper.classification_head.eval()
    viz_model_wrapper.custom_model.eval()

    # 3. 执行推理
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for _, batch in tqdm(viz_test_loader, desc="🔍 正在生成预测"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label_class'].to(device) # 注意：Dataset 中 label 的 key 是 label_class
            
            # 前向传播逻辑 (参考 Classification_model.validation_batch)
            model_outputs = viz_model_wrapper.custom_model(input_ids=input_ids, attention_mask=attention_mask)
            
            # 根据 bottleneck 类型提取特征 (这里假设是 mean 或默认取 [:, 0, :])
            if opts['bottleneck'] == 'mean':
                # 如果是 mean，ModelWithBottleneck 已经处理好了返回
                packet_representation = model_outputs
            else:
                packet_representation = model_outputs[:, 0, :]
            
            label_prob, _ = viz_model_wrapper.classification_head(packet_representation)
            
            preds = torch.argmax(label_prob, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 4. 绘图
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=label_names, yticklabels=label_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix - CIC-IDS2017 NIDS')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # 5. 打印详细报告
    print("\n📝 详细分类报告:")
    print(classification_report(all_labels, all_preds, target_names=label_names))
else:
    print("❌ 错误：未找到 best_weight_path，请先运行上一个单元格。")